## Viés do Observador

### O problema da linha vermelha

- **Linha Vermelha**: é um metrô que conecta Cambridge e Boston;
- Na hora do rush, os trens da Linha Vermelha circulam a **cada 7 a 8 minutos**, em média.
- Estimar o tempo com base na quantidade de passageiros na estação;
    - Poucas pessoas: aproximadamente 7 min;
    - Mais pessoas: mais cedo;
    - Grande número de passageiros: Linha com problema;

### O Modelo

![](intervalos_entre_trens.png)

- z - é a distribuição real;
- zb - é a distribuição tendenciosa vista pelos passageiros.

Considerações:

- Chegadas de passageiros como um processo de Poisson;
    - Mesma probabilidade de chegar a qualquer momento;
    - Cheguem a uma taxa desconhecida, λ, medida em passageiros por minuto(constante);

- O processo de chegada de trens não é Poisson;

Tempo médio entre os trens, visto por um passageiro aleatório:
- Um passageiro é mais provável de chegar durante um intervalo maior;
    - O tempo médio entre os trens é de **7,5 minutos**.
    - O tempo médio entre os trens é de **8,33 minutos**.
    
Viés de observador: 
 - Os alunos pensam que as turmas são maiores;
 - Os passageiros das companhias aéreas acham que os aviões estão mais cheios;
 

In [3]:
def BiasPmf(pmf):
    new_pmf = pmf.Copy()

    for x, p in pmf.Items():
        new_pmf.Mult(x, x)
        
    new_pmf.Normalize()
    return new_pmf

- `pmf` é a distribuição real; 
- `new_pmf` é a distribuição tendenciosa;

### Tempos de espera

![](tempo_de_espera.png)

- **Tempo de espera (y)**: tempo entre a chegada de um passageiro e a próxima chegada de um trem;
- **Tempo de decorrido (x)**: tempo entre a chegada do trem anterior e a chegada de um passageiro;

\begin{equation}
zb = x+ y
\end{equation}

- y É uma mistura de distribuições uniformes ponderadas de acordo com a probabilidade de cada intervalo;

In [6]:
def MakeUniformPmf(low, high):
    pmf = thinkbayes.Pmf()
    for x in MakeRange(low=low, high=high):
        pmf.Set(x, 1)
    pmf.Normalize()
    return pmf

In [9]:
class WaitTimeCalculator(object):

    def __init__(self, pmf_z):
        self.pmf_z = pmf_z
        self.pmf_zb = BiasPmf(pmf)

        self.pmf_y = self.PmfOfWaitTime(self.pmf_zb)
        self.pmf_x = self.pmf_y
        
    def PmfOfWaitTime(pmf_zb):
        metapmf = thinkbayes.Pmf()
        for gap, prob in pmf_zb.Items():
            uniform = MakeUniformPmf(0, gap)
            metapmf.Set(uniform, prob)

        pmf_y = thinkbayes.MakeMixture(metapmf)
        return pmf_y

- A média de z é 7,8 minutos. 
- A média de zb é 8,8 minutos, cerca de 13% maior. 
- A média de y é 4,4, metade da média de zb.

### Prevendo tempos de espera

Suponha que, quando eu chego à plataforma, vejo 10 pessoas esperando. Quanto tempo devo esperar até o próximo trem chegar?

- Distribuição real de z;
- A taxa de chegada de passageiros, λ, é de 2 passageiros por minuto.

Etapas:
- Use a distribuição de z para calcular a **distribuição priori de zp**, o tempo entre os trens, visto pelo passageiro;
- Então, podemos usar o número de passageiros para estimar a distribuição de x, o tempo decorrido desde o último trem.
- Finalmente, usamos a relação y = zp - x para obter a distribuição de y.

In [11]:
class ElapsedTimeEstimator(object):

    def __init__(self, wtc, lam, num_passengers):
        self.prior_x = Elapsed(wtc.pmf_x)

        self.post_x = self.prior_x.Copy()
        self.post_x.Update((lam, num_passengers))

        self.pmf_y = PredictWaitTime(wtc.pmf_zb, self.post_x)

In [15]:
import thinkbayes
class Elapsed(thinkbayes.Suite):

    def Likelihood(self, data, hypo):
        x = hypo
        lam, k = data
        like = thinkbayes.EvalPoissonPmf(k, lam * x)
        return like

A verossimilhança dos dados é a probabilidade de obter k chegadas em x tempo, considerando a taxa de chegada λ.

In [17]:
def PredictWaitTime(pmf_zb, pmf_x):
    pmf_y = pmf_zb - pmf_x
    RemoveNegatives(pmf_y)
    return pmf_y

- O tempo desde o último trem é provavelmente de 5 a 10 minutos.
- O tempo de espera para o próximo trem é de menos de 5 minutos, com cerca de 80% de confiança.

![](prevendo_tempo_de_espera.png)

### Estimando a taxa de chegada

![](lam_pri_pos.png)

Suponha que você acabou de se mudar para Boston, não sabe muito sobre a taxa de chegada de passageiros na Linha Vermelha;

- **k1** é o número de passageiros aguardando quando você chega;
- **y** é o tempo de espera em minutos;
- **k2** é o número de passageiros que chegam enquanto você espera.

k1 | y | k2
---|---|---
17 | 4,6 | 9
22 | 1,0 | 0
23 | 1,4 | 4
18 | 5,4 | 12
4  | 5,8 | 11

- Esperou 18 minutos e viu 36 passageiros chegarem;
    - A taxa de chegada é de 2 passageiros por minuto.

In [19]:
class ArrivalRate(thinkbayes.Suite):

    def Likelihood(self, data, hypo):
        lam = hypo
        y, k = data
        like = thinkbayes.EvalPoissonPmf(k, lam * y)
        return like

In [20]:
class ArrivalRateEstimator(object):

    def __init__(self, passenger_data):
        low, high = 0, 5
        n = 51
        hypos = numpy.linspace(low, high, n) / 60

        self.prior_lam = ArrivalRate(hypos)

        self.post_lam = self.prior_lam.Copy()
        for k1, y, k2 in passenger_data:
            self.post_lam.Update((y, k2))

- A média e a mediana da posterior estão próximas da taxa observada, 2 passageiros por minuto;
- Mas a propagação da distribuição posterior captura nossa incerteza sobre λ com base em uma pequena amostra.

### Incorporando incerteza

![](prevendo_tempo.png)

Sempre que houver incerteza, podemos levar isso em consideração.

- Implemente a análise com base em um valor determinístico do parâmetro incerto (neste caso, λ).
- Calcule a distribuição do parâmetro incerto.
- Execute a análise para cada valor do parâmetro e gere um conjunto de distribuições preditivas.
- Calcule uma mistura das distribuições preditivas, usando os pesos da distribuição do parâmetro.

In [23]:
class WaitMixtureEstimator(object):

    def __init__(self, wtc, are, num_passengers=15):
        self.metapmf = thinkbayes.Pmf()

        for lam, prob in sorted(are.post_lam.Items()):
            ete = ElapsedTimeEstimator(wtc, lam, num_passengers)
            self.metapmf.Set(ete.pmf_y, prob)

        self.mixture = thinkbayes.MakeMixture(self.metapmf)

- Distribuições de y para cada valor de lam;
- A linha escura é a mistura dessas distribuições;

### Análise de decisão

- Usar o número de passageiros na plataforma para prever a distribuição dos tempos de espera.
- Agora vamos à segunda parte da pergunta: quando devo parar de esperar o trem e pegar um táxi?

- A análise é sensível à frequência de longos atrasos (atrasos longos são raros)
- Usar observações anteriores para fazer uma estimativa aproximada.
- Observações são tendenciosas;
    - Devemos tratar as observações como uma amostra de zb em vez de z;

Passos:
- Cria uma amostra de 220 gaps e calculo seu Pmf;

In [1]:
def passs():
    n = 220
    cdf_z = thinkbayes.MakeCdfFromList(gap_times)
    sample_z = cdf_z.Sample(n)
    pmf_z = thinkbayes.MakePmfFromList(sample_z)

- Faz o viés de pmf_z para obter a distribuição de zb;

In [2]:
def passs():
    cdf_zp = BiasPmf(pmf_z).MakeCdf()
    sample_zb = cdf_zp.Sample(n) + [1800, 2400, 3000]

- Usa a amostra de zb para estimar uma PDF usando o KDE;
- Converto a PDF em uma Pmf;

In [4]:
def passs():
    pdf_zb = thinkbayes.EstimatedPdf (sample_zb)
    xs = MakeRange (baixo = 60)
    pmf_zb = pdf_zb.MakePmf (xs)

- Desassocio a distribuição de zb para obter a distribuição de z;

In [3]:
def passs():
    pmf_z = UnbiasPmf(pmf_zb)
    wtc = WaitTimeCalculator(pmf_z)

- Calcular a probabilidade de uma longa espera.

In [5]:
def ProbLongWait(num_passengers, minutes):
    ete = ElapsedTimeEstimator(wtc, lam, num_passengers)
    cdf_y = ete.pmf_y.MakeCdf()
    prob = 1 - cdf_y.Prob(minutes * 60)

![](analise_de_decisão.png)

- Menor que 20: O sistema está operando normalmente;
- Se houver 30 passageiros: algo está errado e esperamos atrasos maiores;

### Discussão

- Taxa de chegada de passageiros é a mesma todos os dias;
- Incluir os eventos especiais no modelo, estendendo a distribuição do lam;
- Suposição de que sabemos a distribuição de z;
- Fazer algumas inferências sobre zb:
    - Estimar o tempo decorrido desde o último trem, x;
    - Observamos y;
    - Adicionarmos a distribuição posterior de x ao y observado;
    - Distribuição posterior sobre o valor observado de zb;
- Atualizar nossas crenças sobre a distribuição do zb;
- Podemos calcular o inverso da distribuição zb e encontrar z;

In [ ]:
- 